<a href="https://colab.research.google.com/github/Pranav-JJ/Transformers-Abstractive-Summarisation/blob/main/ScoreGeneratorSummaryNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement datsets (from versions: none)
ERROR: No matching distribution found for datsets
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


ImportError: ignored

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("phoen1x/T5-Finetuned-legal_summarization")

model = AutoModelForSeq2SeqLM.from_pretrained("phoen1x/T5-Finetuned-legal_summarization", from_tf=True)

All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

All the weights of T5ForConditionalGeneration were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use T5ForConditionalGeneration for predictions without further training.


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.4 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("lighteval/legal_summarization",'BillSum')


test_dataset = dataset['test'][:2000].copy()

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset legal_summarization downloaded and prepared to /root/.cache/huggingface/datasets/lighteval___legal_summarization/BillSum/1.0.0/954313c09b17a47d373856977b2f8a2f26d976f28ded692bb28f2ec4bac2f84c. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(test_dataset['article'][0])

SECTION 1. ENVIRONMENTAL INFRASTRUCTURE. (a) Jackson County, Mississippi.--Section 219 of the Water Resources Development Act of 1992 (106 Stat. 4835; 110 Stat. 3757) is amended-- (1) in subsection (c), by striking paragraph (5) and inserting the following: ``(5) Jackson county, mississippi.--Provision of an alternative water supply and a project for the elimination or control of combined sewer overflows for Jackson County, Mississippi.''; and (2) in subsection (e)(1), by striking ``$10,000,000'' and inserting ``$20,000,000''. (b) Manchester, New Hampshire.--Section 219(e)(3) of the Water Resources Development Act of 1992 (106 Stat. 4835; 110 Stat. 3757) is amended by striking ``$10,000,000'' and inserting ``$20,000,000''. (c) Atlanta, Georgia.--Section 219(f)(1) of the Water Resources Development Act of 1992 (106 Stat. 4835; 113 Stat. 335) is amended by striking ``$25,000,000 for''. (d) Paterson, Passaic County, and Passaic Valley, New Jersey.-- Section 219(f)(2) of the Water Resource

In [ ]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rouge import Rouge

def predict_summary(document):
    device = model.device
    tokenized = tokenizer([document], truncation=True, padding='longest', return_tensors='pt')
    tokenized = {k: v.to(device) for k, v in tokenized.items()}
    tokenized_result = model.generate(**tokenized, max_length=128)
    tokenized_result = tokenized_result.to('cpu')
    predicted_summary = tokenizer.decode(tokenized_result[0])
    return predicted_summary

def calculate_rouge_scores(document, human_summary):

    predicted_summary = predict_summary(document)
    rouge = Rouge()
    scores = rouge.get_scores(predicted_summary, human_summary)
    return scores[0]['rouge-1']['f'], scores[0]['rouge-2']['f'], scores[0]['rouge-l']['f']

In [ ]:
r1=0
r2=0
r3=0

for i in range(10):
  document = test_dataset['article'][i]
  human_summary = test_dataset['summary'][i]

  rouge1_score, rouge2_score, rougel_score = calculate_rouge_scores(document, human_summary)
  print(i)
  r1 = r1+ rouge1_score
  r2 = r2+ rouge2_score
  r3 = r3+ rougel_score




0
1
2
3
4
5
6
7
8
9


In [ ]:
print(r1/10)
print(r2/10)
print(r3/10)

0.3149408926692845
0.17361288906538755
0.2773929147195255
